# Results Analysis Notebook

**EECS 182 Final Project - Analyzing Experiment Results**

This notebook provides deeper analysis of experimental results, including:
- Statistical significance testing
- Spectral trajectory analysis
- Singular value distribution evolution
- Publication-ready figure generation

In [ ]:
# Imports
import sys
import os
sys.path.insert(0, '..')
os.chdir('..')

import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from scipy import stats
import torch

# Set style for publication-quality plots
plt.rcParams.update({
    'font.size': 11,
    'axes.labelsize': 12,
    'axes.titlesize': 13,
    'legend.fontsize': 10,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'figure.figsize': (6, 4),
    'figure.dpi': 150,
    'savefig.dpi': 300,
    'savefig.bbox': 'tight',
    'axes.grid': True,
    'grid.alpha': 0.3,
    'axes.spines.top': False,
    'axes.spines.right': False,
})

print("Analysis notebook initialized.")

In [ ]:
# Load experiment results
try:
    with open('logs/all_experiment_results.json', 'r') as f:
        results = json.load(f)
    print("Loaded experiment results.")
    print(f"Available experiments: {list(results.keys())}")
except FileNotFoundError:
    print("No results file found. Run 02_experiments.ipynb first.")
    results = None

---
## 1. Statistical Analysis: Multi-Seed Comparison

Run multiple seeds to compute confidence intervals.

In [ ]:
# Multi-seed experiment runner
import torch.nn.functional as F
from muon import MuonSGD, SpectralClipSolver, compute_spectral_norms
from models import get_model
import torchvision
import torchvision.transforms as T
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def get_loaders(batch_size=128):
    transform_train = T.Compose([
        T.RandomCrop(32, padding=4),
        T.RandomHorizontalFlip(),
        T.ToTensor(),
        T.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
    ])
    transform_test = T.Compose([
        T.ToTensor(),
        T.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
    ])
    
    train_set = torchvision.datasets.CIFAR10('./data', train=True, download=True, transform=transform_train)
    test_set = torchvision.datasets.CIFAR10('./data', train=False, download=True, transform=transform_test)
    
    return (
        DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2, drop_last=True),
        DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=2)
    )

train_loader, test_loader = get_loaders()

In [ ]:
def run_with_seed(model_name, optimizer_type, inner_solver, lr, epochs, seed, spectral_budget=0.1):
    """Run a single training run with given seed."""
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    
    model = get_model(model_name).to(device)
    
    if optimizer_type == 'muon_sgd':
        from muon import get_inner_solver
        solver = get_inner_solver(inner_solver) if inner_solver != 'none' else None
        optimizer = MuonSGD(
            model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4,
            spectral_budget=spectral_budget if solver else None,
            inner_solver=solver
        )
    else:
        optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
    
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    
    history = []
    for epoch in range(1, epochs + 1):
        # Train
        model.train()
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            loss = F.cross_entropy(model(x), y)
            loss.backward()
            optimizer.step()
        scheduler.step()
        
        # Eval
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for x, y in test_loader:
                x, y = x.to(device), y.to(device)
                correct += (model(x).argmax(1) == y).sum().item()
                total += y.size(0)
        
        val_acc = correct / total
        spec_norms = compute_spectral_norms(model)
        max_spec = max(spec_norms.values()) if spec_norms else 0
        
        history.append({'epoch': epoch, 'val_acc': val_acc, 'max_spec': max_spec})
    
    return history

print("Multi-seed runner ready.")

In [ ]:
# Run multi-seed experiment
NUM_SEEDS = 3  # Increase for better statistics
EPOCHS = 10

configs = [
    ('SGD', 'sgd', 'none'),
    ('MuonSGD', 'muon_sgd', 'spectral_clip'),
]

multi_seed_results = {}

for name, opt, solver in configs:
    print(f"\nRunning {name} across {NUM_SEEDS} seeds...")
    seed_histories = []
    
    for seed in range(NUM_SEEDS):
        print(f"  Seed {seed}...", end=' ')
        history = run_with_seed('small_cnn', opt, solver, lr=0.1, epochs=EPOCHS, seed=seed)
        seed_histories.append(history)
        print(f"Final acc: {history[-1]['val_acc']:.4f}")
    
    multi_seed_results[name] = seed_histories

print("\nMulti-seed experiments complete!")

In [ ]:
# Compute statistics and plot with error bars
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

colors = {'SGD': 'tab:blue', 'MuonSGD': 'tab:orange'}

for name, histories in multi_seed_results.items():
    epochs = [h[0]['epoch'] for h in histories for _ in range(len(histories[0]))]
    epochs = np.array([h['epoch'] for h in histories[0]])
    
    # Stack all seeds
    acc_matrix = np.array([[h['val_acc'] for h in seed_hist] for seed_hist in histories])
    spec_matrix = np.array([[h['max_spec'] for h in seed_hist] for seed_hist in histories])
    
    # Mean and std
    acc_mean = acc_matrix.mean(axis=0)
    acc_std = acc_matrix.std(axis=0)
    spec_mean = spec_matrix.mean(axis=0)
    spec_std = spec_matrix.std(axis=0)
    
    # Plot accuracy
    axes[0].plot(epochs, acc_mean, color=colors[name], label=name, linewidth=2)
    axes[0].fill_between(epochs, acc_mean - acc_std, acc_mean + acc_std,
                         color=colors[name], alpha=0.2)
    
    # Plot spectral norm
    axes[1].plot(epochs, spec_mean, color=colors[name], label=name, linewidth=2)
    axes[1].fill_between(epochs, spec_mean - spec_std, spec_mean + spec_std,
                         color=colors[name], alpha=0.2)

axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Validation Accuracy')
axes[0].set_title('Accuracy (mean ± std over seeds)')
axes[0].legend()

axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Max Spectral Norm')
axes[1].set_title('Spectral Norm (mean ± std over seeds)')
axes[1].legend()

plt.tight_layout()
plt.savefig('logs/multi_seed_comparison.png', dpi=150)
plt.show()

# Statistical test
sgd_final = [h[-1]['val_acc'] for h in multi_seed_results['SGD']]
muon_final = [h[-1]['val_acc'] for h in multi_seed_results['MuonSGD']]

t_stat, p_value = stats.ttest_ind(sgd_final, muon_final)
print(f"\nFinal accuracy comparison (t-test):")
print(f"  SGD:     {np.mean(sgd_final):.4f} ± {np.std(sgd_final):.4f}")
print(f"  MuonSGD: {np.mean(muon_final):.4f} ± {np.std(muon_final):.4f}")
print(f"  t-stat: {t_stat:.3f}, p-value: {p_value:.4f}")

---
## 2. Singular Value Distribution Analysis

Examine how the full singular value spectrum evolves during training.

In [ ]:
# Track full singular value spectrum during training
from muon import compute_all_singular_values

def train_and_track_spectrum(model_name, optimizer_type, inner_solver, epochs=10, seed=42):
    """Train and record singular value spectrum at each epoch."""
    torch.manual_seed(seed)
    model = get_model(model_name).to(device)
    
    if optimizer_type == 'muon_sgd':
        from muon import get_inner_solver
        solver = get_inner_solver(inner_solver)
        optimizer = MuonSGD(
            model.parameters(), lr=0.1, momentum=0.9,
            spectral_budget=0.1, inner_solver=solver
        )
    else:
        optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
    
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    
    # Find a Linear layer to track
    target_layer = None
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Linear):
            target_layer = name
            break
    
    spectrum_history = []
    
    # Initial spectrum
    sv = compute_all_singular_values(model, target_layer)
    if sv is not None:
        spectrum_history.append(sv.cpu().numpy())
    
    for epoch in range(1, epochs + 1):
        model.train()
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            loss = F.cross_entropy(model(x), y)
            loss.backward()
            optimizer.step()
        scheduler.step()
        
        # Record spectrum
        sv = compute_all_singular_values(model, target_layer)
        if sv is not None:
            spectrum_history.append(sv.cpu().numpy())
    
    return spectrum_history, target_layer

print("Tracking SGD spectrum...")
sgd_spectrum, layer_name = train_and_track_spectrum('small_cnn', 'sgd', 'none', epochs=10)

print("Tracking MuonSGD spectrum...")
muon_spectrum, _ = train_and_track_spectrum('small_cnn', 'muon_sgd', 'spectral_clip', epochs=10)

print(f"\nTracked layer: {layer_name}")
print(f"Spectrum length: {len(sgd_spectrum[0])} singular values")

In [ ]:
# Plot singular value evolution
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# Full spectrum at final epoch
axes[0].semilogy(sgd_spectrum[-1], 'b-', label='SGD', linewidth=2)
axes[0].semilogy(muon_spectrum[-1], 'r-', label='MuonSGD', linewidth=2)
axes[0].set_xlabel('Singular Value Index')
axes[0].set_ylabel('Singular Value (log scale)')
axes[0].set_title(f'Final Spectrum ({layer_name})')
axes[0].legend()

# σ_max over time
sgd_sigma_max = [s[0] for s in sgd_spectrum]
muon_sigma_max = [s[0] for s in muon_spectrum]
axes[1].plot(sgd_sigma_max, 'b-', label='SGD', linewidth=2)
axes[1].plot(muon_sigma_max, 'r-', label='MuonSGD', linewidth=2)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('σ_max')
axes[1].set_title('Top Singular Value Evolution')
axes[1].legend()

# Condition number (σ_max / σ_min) over time
sgd_condition = [s[0] / (s[-1] + 1e-10) for s in sgd_spectrum]
muon_condition = [s[0] / (s[-1] + 1e-10) for s in muon_spectrum]
axes[2].semilogy(sgd_condition, 'b-', label='SGD', linewidth=2)
axes[2].semilogy(muon_condition, 'r-', label='MuonSGD', linewidth=2)
axes[2].set_xlabel('Epoch')
axes[2].set_ylabel('Condition Number (log)')
axes[2].set_title('Matrix Conditioning Over Training')
axes[2].legend()

plt.tight_layout()
plt.savefig('logs/singular_value_analysis.png', dpi=150)
plt.show()

print(f"\nFinal condition numbers:")
print(f"  SGD:     {sgd_condition[-1]:.2f}")
print(f"  MuonSGD: {muon_condition[-1]:.2f}")

---
## 3. Inner Solver Convergence Analysis

Analyze how different inner solvers converge to the constraint-satisfying solution.

In [ ]:
# Test inner solver behavior on synthetic gradient
from muon import (
    SpectralClipSolver, FrankWolfeSolver, DualAscentSolver,
    QuasiNewtonDualSolver, ADMMSolver
)

# Create synthetic test case
torch.manual_seed(42)
m, n = 64, 128
W = torch.randn(m, n)  # Weight matrix
G = torch.randn(m, n) * 0.5  # Gradient (proposed update)

budget = 0.1

print(f"Test case: W shape = {W.shape}, budget = {budget}")
print(f"Original ||G||_2 = {torch.linalg.matrix_norm(G, ord=2).item():.4f}")

solvers = {
    'SpectralClip': SpectralClipSolver(),
    'FrankWolfe (5 iter)': FrankWolfeSolver(max_iters=5),
    'FrankWolfe (20 iter)': FrankWolfeSolver(max_iters=20),
    'DualAscent': DualAscentSolver(max_iters=20),
    'QuasiNewton': QuasiNewtonDualSolver(max_iters=15),
    'ADMM': ADMMSolver(max_iters=20),
}

print("\nSolver results:")
print("-" * 50)
for name, solver in solvers.items():
    result = solver(W, G, budget)
    sigma = torch.linalg.matrix_norm(result, ord=2).item()
    obj = torch.sum(G * result).item()  # Linear objective
    print(f"{name:25s}: ||Δ||_2 = {sigma:.4f}, <G,Δ> = {obj:.4f}")

In [ ]:
# Visualize solver outputs
fig, axes = plt.subplots(2, 3, figsize=(12, 8))

for ax, (name, solver) in zip(axes.flatten(), solvers.items()):
    result = solver(W, G, budget)
    
    # Show singular values of the result
    sv = torch.linalg.svdvals(result).numpy()
    ax.bar(range(len(sv)), sv, alpha=0.7)
    ax.axhline(budget, color='r', linestyle='--', label=f'Budget = {budget}')
    ax.set_xlabel('Singular Value Index')
    ax.set_ylabel('Singular Value')
    ax.set_title(f'{name}')
    ax.set_xlim(-1, 20)  # Show first 20
    ax.legend(fontsize=8)

plt.tight_layout()
plt.savefig('logs/inner_solver_spectra.png', dpi=150)
plt.show()

---
## 4. Publication-Ready Figures

Generate final figures for the project report.

In [ ]:
# Main comparison figure (2x2 layout)
if results is not None and 'solver_comparison' in results:
    fig, axes = plt.subplots(2, 2, figsize=(10, 8))
    
    solver_data = results['solver_comparison']
    colors = plt.cm.Set2(np.linspace(0, 1, len(solver_data)))
    
    for (name, history), color in zip(solver_data.items(), colors):
        epochs = [h['epoch'] for h in history]
        
        # Val loss
        axes[0, 0].plot(epochs, [h['val_loss'] for h in history], 
                        label=name, color=color, linewidth=1.5)
        # Val acc
        axes[0, 1].plot(epochs, [h['val_acc'] for h in history],
                        label=name, color=color, linewidth=1.5)
        # Spectral norm
        axes[1, 0].plot(epochs, [h['max_spectral_norm'] for h in history],
                        label=name, color=color, linewidth=1.5)
        # Sharpness
        axes[1, 1].plot(epochs, [h['sharpness'] for h in history],
                        label=name, color=color, linewidth=1.5)
    
    axes[0, 0].set_ylabel('Validation Loss')
    axes[0, 0].set_title('(a) Validation Loss')
    
    axes[0, 1].set_ylabel('Validation Accuracy')
    axes[0, 1].set_title('(b) Validation Accuracy')
    
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Max Spectral Norm')
    axes[1, 0].set_title('(c) Spectral Norm Control')
    
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Sharpness')
    axes[1, 1].set_title('(d) Loss Landscape Sharpness')
    
    # Single legend
    handles, labels = axes[0, 0].get_legend_handles_labels()
    fig.legend(handles, labels, loc='upper center', ncol=4, bbox_to_anchor=(0.5, 1.02))
    
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.savefig('logs/figure_main_comparison.pdf', dpi=300)
    plt.savefig('logs/figure_main_comparison.png', dpi=300)
    plt.show()
    print("Saved: logs/figure_main_comparison.pdf")
else:
    print("Run experiments first to generate this figure.")

In [ ]:
# Width transfer figure
if results is not None and 'width_transfer_muon' in results:
    fig, ax = plt.subplots(figsize=(8, 5))
    
    muon_data = results['width_transfer_muon']
    sgd_data = results['width_transfer_sgd']
    
    widths = sorted([float(w) for w in muon_data.keys()])
    muon_accs = [muon_data[str(w)][-1]['val_acc'] for w in widths]
    sgd_accs = [sgd_data[str(w)][-1]['val_acc'] for w in widths]
    
    ax.plot(widths, muon_accs, 'o-', label='MuonSGD (spectral constraint)',
            markersize=12, linewidth=2.5, color='tab:blue')
    ax.plot(widths, sgd_accs, 's--', label='SGD (baseline)',
            markersize=10, linewidth=2, color='tab:orange')
    
    ax.set_xlabel('Width Multiplier', fontsize=12)
    ax.set_ylabel('Final Validation Accuracy', fontsize=12)
    ax.set_title('Hyperparameter Transfer Across Widths', fontsize=14)
    ax.legend(fontsize=11)
    
    # Add variance annotation
    muon_std = np.std(muon_accs)
    sgd_std = np.std(sgd_accs)
    ax.annotate(f'MuonSGD σ = {muon_std:.4f}\nSGD σ = {sgd_std:.4f}',
                xy=(0.95, 0.05), xycoords='axes fraction',
                ha='right', va='bottom',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.tight_layout()
    plt.savefig('logs/figure_width_transfer.pdf', dpi=300)
    plt.savefig('logs/figure_width_transfer.png', dpi=300)
    plt.show()
    print("Saved: logs/figure_width_transfer.pdf")
else:
    print("Run experiments first to generate this figure.")

---
## 5. Results Summary Table

In [ ]:
# Generate results summary table
if results is not None and 'solver_comparison' in results:
    summary_data = []
    
    for name, history in results['solver_comparison'].items():
        final = history[-1]
        best_acc = max(h['val_acc'] for h in history)
        final_spec = final['max_spectral_norm']
        final_sharp = final['sharpness']
        
        summary_data.append({
            'Method': name,
            'Final Val Acc': f"{final['val_acc']:.4f}",
            'Best Val Acc': f"{best_acc:.4f}",
            'Final σ_max': f"{final_spec:.3f}",
            'Final Sharpness': f"{final_sharp:.4f}",
        })
    
    df = pd.DataFrame(summary_data)
    print("\nResults Summary (ResNet-18 on CIFAR-10):")
    print("=" * 80)
    print(df.to_string(index=False))
    
    # Save to CSV
    df.to_csv('logs/results_summary.csv', index=False)
    print("\nSaved: logs/results_summary.csv")
else:
    print("Run experiments first to generate summary.")

---
## 6. Key Takeaways

### Observations from Experiments:

1. **Spectral Norm Control**: MuonSGD effectively controls the maximum singular values of weight matrices during training, while vanilla SGD allows unbounded growth.

2. **Inner Solver Trade-offs**:
   - **SpectralClip**: Fastest, but may distort gradient direction
   - **DualAscent**: Good balance of accuracy and speed
   - **QuasiNewton**: Faster convergence on the dual, but more complex
   - **FrankWolfe**: Produces low-rank updates, projection-free
   - **ADMM**: Clean separation of constraints, adaptive ρ helps

3. **Width Transfer**: Spectral constraints reduce variance in final accuracy across different model widths, supporting better hyperparameter transfer.

4. **Stability**: MuonSGD may extend the stable learning rate region compared to vanilla SGD.

### Connections to Theory:

- Spectral norm constraints provide a **Lipschitz bound** on layer-wise transformations
- Lower sharpness correlates with the **flatness of minima** hypothesis
- Width-independent hyperparameters align with **muP scaling** principles

### Future Work:

- Extend to larger-scale transformers
- Combine with manifold constraints (Stiefel)
- Investigate connections to implicit regularization